# OpenAI Responses API

## What is the OpenAI Responses API?

The Responses API is a new API released in March 2025. It is a combination of the traditional 
Chat Completions API and the Assistants API, providing support for:

- **Traditional Chat Completions:** Facilitates seamless conversational AI experiences.
- **Web Search:** Enables real-time information retrieval from the internet.
- **File Search:** Allows searching within files for relevant data.

Accordingly, the Assistants API will be retired in 2026. 

> **For new users, OpenAI recommends using the Responses API instead of the Chat Completions API to leverage its expanded capabilities.**

For a comprehensive comparison between the Responses API and the Chat Completions API, refer to the official OpenAI documentation: 
[Responses vs. Chat Completions](https://platform.openai.com/docs/guides/responses-vs-chat-completions).

## Summary of This Notebook
This notebook provides a hands-on guide for using the **OpenAI Responses API** to analyze tweets. 
It covers essential techniques such as:

- **Creating a vector store** and uploading tweets for semantic search.
- **Using file search** to analyze private datasets.
- **Performing a web search** to retrieve the latest public information.
- **Utilizing stateful responses** to maintain conversation context.
- **Combining file and web search** to enhance retrieval-augmented generation (RAG) applications.

By the end of this notebook, users will be able to integrate OpenAI's Responses API for efficient data retrieval and analysis of structured and unstructured data.

## Install Required Libraries
To use the OpenAI Responses API, we need to install the following libraries:

- **`openai`**: Provides access to OpenAI's APIs, including the Responses API

In [1]:
pip install openai -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sparkmagic 0.21.0 requires pandas<2.0.0,>=0.17.1, but you have pandas 2.3.3 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


## Import Required Libraries

In [2]:
from IPython.display import Markdown, display
import boto3
from botocore.exceptions import ClientError
import json
import io

## Retrieve Secrets from AWS Secrets Manager

In [3]:
def get_secret(secret_name):
    region_name = "us-east-1"

    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        raise e

    secret = get_secret_value_response['SecretString']
    
    return json.loads(secret)

## Initialize OpenAI Client

In [4]:
from openai import OpenAI
openai_api_key  = get_secret('openai')['api_key']

client = OpenAI(api_key=openai_api_key)

## File Search API

### Introduction to File Search
File search API enables efficient retrieval of relevant information 
from uploaded files by leveraging vector-based indexing. This feature is particularly useful 
for searching large datasets, extracting insights, and improving retrieval-augmented generation (RAG) applications.

Unlike traditional keyword-based searches, the Responses API uses embeddings 
to identify semantically relevant content, making it ideal for analyzing structured 
and unstructured text data (OpenAI, 2025).

For more details, visit the official OpenAI documentation: 
[File Search in Responses API](https://platform.openai.com/docs/guides/tools-file-search).

### Create a Vector Store

In [5]:
vector_store = client.vector_stores.create(
    name="my_vector_store"
)
vector_store_id = vector_store.id
print(vector_store_id)

vs_6914ccd04e68819188fed341eb13f65a


### Upload Files

In [6]:
with open('tweet_text.json', 'rb') as f:
    file = client.files.create(
        file=f,            # file-like object
        purpose="assistants"
    )

file_id = file.id
print(file_id)

file-D5yJBBuGB258mEAmEh1on3


### Attach File to Vector Store

In [7]:
attach_status =client.vector_stores.files.create(
    vector_store_id=vector_store_id,
    file_id=file_id
            )

print(attach_status.id)

file-D5yJBBuGB258mEAmEh1on3


### Query the Vector Store

In [8]:
query = "the latest development in generativeAI"

In [9]:
search_results = client.vector_stores.search(
    vector_store_id=vector_store_id,
    query=query
)

for result in search_results.data[:5]:
    print(result.content[0].text[:100] + '\n Relevant score: ' + str(result.score))

They have been VERY clear that Chatgpt, these AI videos…"
  }
},
{
  "_id": {
    "$oid": "68e56b1e1
 Relevant score: 0.6760340526672118
They have been VERY clear that Chatgpt, these AI videos…"
  }
},
{
  "_id": {
    "$oid": "68e56b1f1
 Relevant score: 0.6010846063542131
Learning Plans. Use coupon code 𝐒𝐏𝐋𝟑𝟎 at checkout.🎯\n\nJoin Now 👉 https://t.co/LS2JuCrVmz\n\n#AI #Ce
 Relevant score: 0.5984074448147717
Create stunning, cinematic videos from a prompt—now with audio, physics, and cameos. One prompt = en
 Relevant score: 0.591898195864025
⚡\n\nThe global #GenerativeAI market will hit $1.18 B this year — act now or get left behind. 🚀\n\n🔗
 Relevant score: 0.5731275237390817


## OpenAI Response API

### Simple Response

In [10]:
simple_response = client.responses.create(
  model="gpt-4o",
  input=[
      {
          "role": "user",
          "content": query
      }
  ]
)

In [11]:
display(Markdown(simple_response.output_text))

As of my last update, there have been several exciting developments in generative AI:

1. **Foundation Models**: Companies are focusing on building large foundation models that can be fine-tuned for various tasks. These models are pre-trained on vast datasets, making them versatile across numerous applications.

2. **Multi-Modal AI**: There is an increasing emphasis on models that can handle multiple types of data, such as text, images, and audio simultaneously, enhancing their ability to interact in more human-like ways.

3. **Improved Efficiency**: New architectures and training techniques aim to reduce the computational resources required, making AI models more accessible and environmentally friendly.

4. **Applications in Creative Industries**: Generative AI is being used to assist in music production, art generation, video creation, and more, offering new tools for creativity.

5. **Ethical and Responsible AI**: There's a stronger focus on ensuring that AI systems are fair, unbiased, and transparent. Research is being directed toward understanding and mitigating biases within AI models.

6. **Regulation and Policy**: Governments and organizations are working on regulations to oversee the use and development of AI to ensure it aligns with societal values and legal standards.

These developments are shaping the future trajectory of AI, making it more integrated and impactful across various sectors.

### File Search Response

In [12]:

file_search_response = client.responses.create(
    input= query,
    model="gpt-4o",
    temperature = 0,
    tools=[{
        "type": "file_search",
        "vector_store_ids": [vector_store_id],
    }]
)

In [13]:
display(Markdown(file_search_response.output_text))


The latest developments in generative AI include:

1. **OpenAI's Sora2**: This tool allows users to create cinematic videos from a prompt, now with added features like audio, physics, and cameos.

2. **Generative AI in Business**: AI agents are evolving beyond chatbots to become autonomous, collaborative teammates that are specialized, multimodal, and emotionally intelligent.

3. **Generative AI in Creative Industries**: AI is transforming creative fields by enabling the creation of content like cinematic food commercials without the need for traditional production setups.

4. **Enterprise Applications**: IBM's Watsonx is bringing generative AI to enterprises with tools for secure and responsible deployment, allowing for custom large language models.

These advancements highlight the growing impact of generative AI across various sectors, from media and entertainment to enterprise solutions.

## Web Search API

### Introduction to Web Search
The OpenAI Web Search tool allows models to retrieve real-time information from the internet. 
This capability is particularly useful for obtaining up-to-date data, fact-checking, and expanding knowledge 
without relying solely on pre-trained information. 

By leveraging OpenAI's web search functionality, the Responses API can fetch external data 
and provide accurate, relevant results in real time (OpenAI, 2025). 
This feature enhances applications that require the latest insights, such as news aggregation, research, 
or dynamic content generation.

For more details, visit the official OpenAI documentation: 
[Web Search in Responses API](https://platform.openai.com/docs/guides/tools-web-search).

### Perform Web Search

In [14]:
web_search_response = client.responses.create(
    model="gpt-4o",  # or another supported model
    input= query,
    tools=[
        {
            "type": "web_search"
        }
    ]
)

In [15]:
display(Markdown(web_search_response.output_text))

Here’s a deep, structured, and up-to-date overview of the latest developments in generative AI as of **November 12, 2025**, with richly cited sources to ensure accuracy and reliability:

---

## 1. Industry Trends & Economic Impact

• **Rapid Productivity Gains & Unequal Adoption**  
Generative AI is being adopted faster than previous technologies, showing meaningful productivity boosts—e.g., a 55% gain in software coding tasks—but usage remains uneven, with younger, more educated individuals benefitting most ([voxeu.org](https://voxeu.org/voxeu/columns/generative-ai-uneven-adoption-labour-market-returns-and-policy-implications?utm_source=openai)).

• **Market Growth & Applications in HR**  
The generative AI market has surged to an estimated **$66.89 billion in 2025**, with expanding applications across fields like human resources ([aijourn.com](https://aijourn.com/generative-ai-in-hrtech-opportunities-risks-and-future/?utm_source=openai)).

---

## 2. Major Corporate Partnerships

• **Credem & Google Cloud Extend AI Tools in Banking**  
On **November 11, 2025**, Italy’s Credem announced an extended partnership with Google Cloud to integrate Gemini-powered generative AI tools into its employee productivity workflows ([prnewswire.com](https://www.prnewswire.com/news-releases/credem-strengthens-partnership-with-google-cloud-to-accelerate-innovation-with-generative-ai-302609913.html?utm_source=openai)).

---

## 3. Geopolitical Position & Intellectual Property

• **China Leads in Generative AI Patents**  
Between 2014 and 2023, China held the most patents in generative AI, underscoring its strong positional leadership alongside the U.S. in global AI development ([globaltimes.cn](https://www.globaltimes.cn/page/202511/1347976.shtml?utm_source=openai)).

---

## 4. Advancements in Media Generation

• **Google Veo 3.1: Enhanced Control for Creators**  
In October 2025, DeepMind launched **Veo 3.1** in paid preview via the Gemini API. It introduces extended narrative control—like seamless clip extensions using first and last frames, improved character consistency, and built-in audio—with tight Flow app integration ([voxfor.com](https://www.voxfor.com/what-is-new-in-ai-the-latest-news-from-october-2025/?utm_source=openai)).

• **Runway Gen‑4 Video Model**  
Released March 2025, Runway’s **Gen-4** model generates up to 10-second video clips from text or reference images. It uses transformer-based diffusion, supports long prompts, and serves sectors like film, advertising, and previsualization ([en.wikipedia.org](https://en.wikipedia.org/wiki/Gen-4_%28AI_image_and_video_model%29?utm_source=openai)).

• **Google “Nano Banana” (Gemini 2.5 Flash Image)**  
Introduced in August 2025, this advanced image editing model enables highly consistent and photorealistic edits—allowing changes like hairstyle or backdrop while preserving subject identity. It supports multi-image fusion and features SynthID watermarking for content tracking ([en.wikipedia.org](https://en.wikipedia.org/wiki/Gemini_%28language_model%29?utm_source=openai)).

---

## 5. Broader Ecosystem & Tool Innovations

• **Game Development & Generative AI Adoption**  
Nexon’s CEO emphasized that “every game company is now using AI,” reflecting how generative techniques (e.g., voice acting, content creation) have become integral to game development ([automaton-media.com](https://automaton-media.com/en/news/its-important-to-assume-every-game-company-is-now-using-ai-nexon-ceo-comments-on-role-of-human-creativity-as-ai-tools-become-the-norm/?utm_source=openai)).

• **Educational Usage & Acceptance**  
A study published **today** investigated factors influencing university students’ acceptance of generative AI. It found that higher AI literacy and relevant technology exposure significantly improve adoption ([bmcpsychology.biomedcentral.com](https://bmcpsychology.biomedcentral.com/articles/10.1186/s40359-025-03559-2?utm_source=openai)).

---

## 6. Summary Table of Notable Developments

| Area                      | Development Highlights |
|---------------------------|------------------------|
| Productivity & Equity     | 55% coding gains; uneven usage ([voxeu.org](https://voxeu.org/voxeu/columns/generative-ai-uneven-adoption-labour-market-returns-and-policy-implications?utm_source=openai)) |
| Market Growth             | Generative AI worth US$66.89B ([aijourn.com](https://aijourn.com/generative-ai-in-hrtech-opportunities-risks-and-future/?utm_source=openai)) |
| Industry Partnerships     | Credem integrates Gemini AI tools ([prnewswire.com](https://www.prnewswire.com/news-releases/credem-strengthens-partnership-with-google-cloud-to-accelerate-innovation-with-generative-ai-302609913.html?utm_source=openai)) |
| Patent Leadership         | China leads generative AI patents ([globaltimes.cn](https://www.globaltimes.cn/page/202511/1347976.shtml?utm_source=openai)) |
| Video Generation          | Veo 3.1 enables narrative control ([voxfor.com](https://www.voxfor.com/what-is-new-in-ai-the-latest-news-from-october-2025/?utm_source=openai)) |
| Media Tools               | Runway Gen‑4 enables video generation ([en.wikipedia.org](https://en.wikipedia.org/wiki/Gen-4_%28AI_image_and_video_model%29?utm_source=openai)) |
| Image Consistency         | Nano Banana preserves identity ([en.wikipedia.org](https://en.wikipedia.org/wiki/Gemini_%28language_model%29?utm_source=openai)) |
| Game & AI Integration     | Generative AI widely used in gaming ([automaton-media.com](https://automaton-media.com/en/news/its-important-to-assume-every-game-company-is-now-using-ai-nexon-ceo-comments-on-role-of-human-creativity-as-ai-tools-become-the-norm/?utm_source=openai)) |
| AI in Higher Education    | Student acceptance tied to AI literacy ([bmcpsychology.biomedcentral.com](https://bmcpsychology.biomedcentral.com/articles/10.1186/s40359-025-03559-2?utm_source=openai)) |

---

## Context & Relevance

- **Innovations in media generation**—namely Google’s Veo 3.1 and Nano Banana—indicate a shift toward more controllable and consistent generative outputs, addressing critical challenges in AI creativity (e.g., character stability, narrative flow).
- **Corporate adoption** (e.g., Credem) highlights how generative AI is no longer just experimental—it’s a strategic productivity tool in enterprise settings.
- **Geopolitical dynamics**—China’s patent leadership—reflect growing international stakes in AI R&D.
- **Economic gains** and **digital equity issues** underscore the need for inclusive AI development and deployment.
- **Educational research** suggests that building AI literacy is critical to foster responsible and widespread AI usage.

---

Let me know if you’d like to explore any of these developments in more depth—such as technical architecture, comparative model capabilities, ethical considerations, or adoption strategies in specific industries.

### Stateful Response

The OpenAI Responses API includes a stateful feature that enables continuity in interactions. 
By using the `response_id`, a conversation can persist across multiple queries, 
allowing users to refine or expand upon previous searches. This is particularly useful for iterative research, 
dynamic content generation, and applications that require follow-up queries based on prior responses.

In [16]:
fetched_response = client.responses.retrieve(response_id=web_search_response.id)
display(Markdown(fetched_response.output_text[:100]))

Here’s a deep, structured, and up-to-date overview of the latest developments in generative AI as of

### Continue Query with Web Search

In [17]:
continue_query = 'find different news'

continue_search_response = client.responses.create(
    model="gpt-4o",  # or another supported model
    input= continue_query,
    previous_response_id=web_search_response.id,
    tools=[
        {
            "type": "web_search"
        }
    ]
)

In [18]:
display(Markdown(continue_search_response.output_text))

Below are several **recent key developments in generative AI**—all reported **today or yesterday (November 11–12, 2025)**—organized thematically with precise dates and citations.

**1. AI Policy & Legal Affairs**  
- A U.S. federal judge has allowed a **copyright infringement lawsuit** against OpenAI to proceed, involving authors who claim that ChatGPT’s text summaries infringe their rights. The Publishers Association highlighted how generative AI can severely impact creative markets. ([thebookseller.com](https://www.thebookseller.com/news/openai-copyright-case-reveals-ease-with-which-generative-ai-can-devastate-the-market-says-pa?utm_source=openai))

**2. AI in Consumer Tools & Media**  
- **Google Photos** now includes six new AI-powered editing tools, several of which use the “Nano Banana” image model—Google’s Gemini 2.5 Flash Image system. ([petapixel.com](https://petapixel.com/2025/11/11/generative-ai-gemini-nano-banana-is-now-inside-google-photos/?utm_source=openai))  
- Fei‑Fei Li’s startup **World Labs** unveiled **Marble**, the first commercial platform capable of converting text, images, and video into persistent, editable 3D environments—marking a major step toward spatially-aware generative AI. ([cryptorank.io](https://cryptorank.io/news/feed/fe70b-fei-fei-li-marble-world-models?utm_source=openai))

**3. AI Infrastructure & Research**  
- The **University of Texas at Austin** has **doubled the capacity** of its AI research infrastructure, now hosting over **1,000 advanced GPUs** dedicated to generative AI work. ([quantumzeitgeist.com](https://quantumzeitgeist.com/ut-ai-computing/?utm_source=openai))  
- **Georgia Tech’s Cloud Hub**, powered by Microsoft funding, recently completed a Call for Proposals to support foundational generative AI research and innovative applications. ([newswise.com](https://www.newswise.com/articles/georgia-tech-cloud-hub-advances-generative-ai-research-with-microsoft-support?utm_source=openai))

**4. Business Trends & Market Insights**  
- According to Morningstar, **62% of marketers** recognize the trend of “click-less journeys” in online experiences—but just **27%** feel prepared for these AI-driven shifts. ([morningstar.com](https://www.morningstar.com/news/pr-newswire/20251112ny21452/ai-discovery-has-rewritten-the-rules-of-online-shopping-and-research-most-brands-are-underprepared?utm_source=openai))  
- Market coverage indicates that AI-related optimism is reshaping investor sentiment, propelling equity indices to new heights as generative technologies become growth drivers. ([markets.financialcontent.com](https://markets.financialcontent.com/wral/article/marketminute-2025-11-12-the-ai-ascent-how-hopes-and-hype-are-reshaping-the-2025-stock-market?utm_source=openai))  
- Microsoft, conversely, has downplayed fears of an “AI bubble,” asserting that sustained spending and investment in AI is here to stay. ([computerworld.com](https://www.computerworld.com/article/4087593/about-that-ai-bubble-microsoft-doesnt-see-one.html?utm_source=openai))

**5. Innovations in AI Applications & Research**  
- A new project—**Unheard**—aims to help news organizations surface overlooked narratives by using AI to audit sources and identify who is quoted in their reporting. ([cjr.org](https://www.cjr.org/tow_center/toward-ai-powered-source-audits.php?utm_source=openai))  
- In **Brazil**, the **2025 ISG Provider Lens® Generative AI Services report** evaluated **43 providers**, listing Accenture, IBM, and Stefanini among the leaders as generative AI shifts from pilot projects to production. ([stocktitan.net](https://www.stocktitan.net/news/III/gen-ai-plays-growing-role-in-brazilian-w336ew2mzxxn.html?utm_source=openai))  
- **Insilico Medicine** will showcase its new generative AI platform and research in pulmonary fibrosis at the **Pulmonary Fibrosis Foundation Summit** in Chicago (November 13–15, 2025). ([bioengineer.org](https://bioengineer.org/insilico-medicine-to-unveil-generative-ai-platform-and-cutting-edge-ai-driven-pulmonary-fibrosis-research-at-pff-summit-2025-in-chicago/?utm_source=openai))

---

###  Summary Table of Latest Advances

| Date         | Highlight                                                                 |
|--------------|---------------------------------------------------------------------------|
| Nov 12, 2025 | OpenAI copyright case proceeds ([thebookseller.com](https://www.thebookseller.com/news/openai-copyright-case-reveals-ease-with-which-generative-ai-can-devastate-the-market-says-pa?utm_source=openai))                         |
| Nov 11, 2025 | Nano Banana features added to Google Photos ([petapixel.com](https://petapixel.com/2025/11/11/generative-ai-gemini-nano-banana-is-now-inside-google-photos/?utm_source=openai)) |
| Nov 12, 2025 | Launch of Marble by Fei‑Fei Li's World Labs ([cryptorank.io](https://cryptorank.io/news/feed/fe70b-fei-fei-li-marble-world-models?utm_source=openai))           |
| Today        | UT Austin doubles AI infrastructure ([quantumzeitgeist.com](https://quantumzeitgeist.com/ut-ai-computing/?utm_source=openai))                     |
| Today        | Georgia Tech Cloud Hub’s GenAI research initiative ([newswise.com](https://www.newswise.com/articles/georgia-tech-cloud-hub-advances-generative-ai-research-with-microsoft-support?utm_source=openai))      |
| Today        | Marketers unprepared for click-less journeys ([morningstar.com](https://www.morningstar.com/news/pr-newswire/20251112ny21452/ai-discovery-has-rewritten-the-rules-of-online-shopping-and-research-most-brands-are-underprepared?utm_source=openai))            |
| Today        | AI optimism fuels markets ([markets.financialcontent.com](https://markets.financialcontent.com/wral/article/marketminute-2025-11-12-the-ai-ascent-how-hopes-and-hype-are-reshaping-the-2025-stock-market?utm_source=openai))                              |
| Today        | Microsoft denies AI bubble concerns ([computerworld.com](https://www.computerworld.com/article/4087593/about-that-ai-bubble-microsoft-doesnt-see-one.html?utm_source=openai))                    |
| Today        | Unheard audits news sources via AI ([cjr.org](https://www.cjr.org/tow_center/toward-ai-powered-source-audits.php?utm_source=openai))                     |
| Today        | ISG report on Brazil’s GenAI providers ([stocktitan.net](https://www.stocktitan.net/news/III/gen-ai-plays-growing-role-in-brazilian-w336ew2mzxxn.html?utm_source=openai))                |
| Today        | Insilico's AI platform featured at PFF Summit ([bioengineer.org](https://bioengineer.org/insilico-medicine-to-unveil-generative-ai-platform-and-cutting-edge-ai-driven-pulmonary-fibrosis-research-at-pff-summit-2025-in-chicago/?utm_source=openai))         |

---

If you'd like a deeper dive into any of these stories—such as legal ramifications, technical details of Marble or Nano Banana, academic impacts of UT's GPU expansion, sector comparisons, or implications for the AI job market—just let me know!

### Combining File Search and Web Search

This is an example of using file search to analyze private data and web search to retrieve public or the latest data. 
The Responses API allows developers to integrate these tools to enhance retrieval-augmented generation (RAG) applications. 
By combining file search with web search, users can leverage structured internal knowledge while also retrieving real-time 
information from external sources, ensuring comprehensive and up-to-date responses. 

In [19]:
combined_search_response = client.responses.create(
    model="gpt-4o",  # or another supported model
    input= query,
    temperature = 0,
    instructions="Retrieve the results from the file search first, and use the web search tool to expand the results with news resources",
    tools=[{
        "type": "file_search",
        "vector_store_ids": [vector_store_id],
    },
        {
            "type": "web_search"
        }
    ]
)

In [20]:
display(Markdown(combined_search_response.output_text))

Recent developments in generative AI include several exciting advancements:

1. **AI Video Creation**: OpenAI's Sora2 is now live, allowing users to create cinematic videos from a prompt, complete with audio, physics, and cameos. This tool is seen as a game changer for creators.

2. **Enterprise Applications**: IBM's Watsonx is bringing generative AI to enterprises with tools designed for secure and responsible deployment. This allows teams to build custom large language models to enhance customer engagement and streamline processes.

3. **Creative Industries**: Generative AI is significantly impacting creative industries, enabling the creation of content like music and videos without traditional production resources.

4. **Healthcare**: In healthcare, generative AI is being used to support doctors with data interpretation and preliminary analysis, highlighting its potential to transform patient care.

These developments illustrate the broadening scope and application of generative AI across various sectors, from media and entertainment to enterprise solutions and healthcare.

# 🧩 Try It Yourself: Two-Step RAG (Private Data + Combined Search)

## Step 1 — Upload & Create Vector Store
1. Upload a short text file (e.g., `my_notes.txt`) to your notebook instance.  
2. Create a **vector store** and **ingest** your uploaded file.  
3. Run a simple test query to verify retrieval:  

In [23]:
with open('news.txt', 'rb') as f:
    file = client.files.create(
        file=f,            # file-like object
        purpose="assistants"
    )

file_id = file.id
print(file_id)

file-GU22DxadfKrgatLZy5A6uR


In [24]:
attach_status =client.vector_stores.files.create(
    vector_store_id=vector_store_id,
    file_id=file_id
            )

print(attach_status.id)

file-GU22DxadfKrgatLZy5A6uR


In [25]:
query = "Who is leading in the 2025 Virginia governor's race?"

In [26]:
search_results = client.vector_stores.search(
    vector_store_id=vector_store_id,
    query=query
)

for result in search_results.data[:5]:
    print(result.content[0].text[:100] + '\n Relevant score: ' + str(result.score))

WTVR CBS 6 News Richmond
Watch Now 

WTVR CBS 6 News Richmond
Watch Now 
×
Search site
Search
Go
WTV
 Relevant score: 0.9870768634974457
"We didn't come this far to only go this far," Earle-Sears told supporters. "We have done so much go
 Relevant score: 0.5435440078655142
📱More Virginia election news from WTVR.com
 Earle-Sears, Spanberger make final push as Election Day 
 Relevant score: 0.3600680955077078
They have been VERY clear that Chatgpt, these AI videos…"
  }
},
{
  "_id": {
    "$oid": "68e56b1e1
 Relevant score: 0.07636730368401264
They have been VERY clear that Chatgpt, these AI videos…"
  }
},
{
  "_id": {
    "$oid": "68e56b1d1
 Relevant score: 0.07384093233110427


## Step 2 — Combine File Search with Web Search
1. Enable both **file_search** and **web_search** in the Responses API.  
2. Use a prompt that asks the model to merge insights from both sources.  
   > Example: “Using my uploaded notes and the latest web information, summarize the current trends on this topic.”  
3. Review how the answer from your file and **current info** from the web.

✅ You’ve created a RAG system that combines **private** and **public** data for comprehensive, up-to-date analysis.


In [27]:
file_search_response = client.responses.create(
    input= query,
    model="gpt-4o",
    temperature = 0,
    tools=[{
        "type": "file_search",
        "vector_store_ids": [vector_store_id],
    }]
)

In [29]:
display(Markdown(file_search_response.output_text))

Democrat Abigail Spanberger is leading Republican Winsome Earle-Sears in the 2025 Virginia governor's race. According to a recent poll, Spanberger is ahead with 55% compared to Earle-Sears' 44%.

In [30]:
web_search_response = client.responses.create(
    model="gpt-4o",  # or another supported model
    input= query,
    tools=[
        {
            "type": "web_search"
        }
    ]
)

In [31]:
display(Markdown(web_search_response.output_text))

Here’s the latest on the 2025 Virginia governor’s race (as of **November 12, 2025**):

Recent polls and aggregated data consistently show Democrat **Abigail Spanberger** maintaining a substantial lead over Republican Lt. Gov. **Winsome Earle‑Sears**.

Poll Highlights:

- A **Virginia Commonwealth University (VCU) Commonwealth Poll**, conducted October 6–14 and released October 21, found Spanberger leading Earle‑Sears by **49% to 42%**, a 7-point margin—down slightly from earlier double-digit leads ([virginiamercury.com](https://virginiamercury.com/2025/10/21/spanbergers-lead-narrows-as-virginia-races-tighten-in-new-vcu-poll/?utm_source=openai)).

- The **Washington Post–Schar School poll**, conducted around early October, showed Spanberger ahead by **12 percentage points**, with 55% support to Earle‑Sears’ 43% among likely voters, and similar margins among registered voters ([washingtonpost.com](https://www.washingtonpost.com/dc-md-va/2025/10/03/virginia-election-poll-spanberger-earle-sears//?utm_source=openai)).

- **Emerson College Polling/The Hill**, around early October, reported Spanberger at **52%**, Earle‑Sears at **42%**, for a 10-point advantage ([emersoncollegepolling.com](https://emersoncollegepolling.com/virginia-2025-polling/?utm_source=openai)).

- The **Roanoke College Poll** from late October indicated a **10-point margin**—Spanberger at 51% vs. Earle‑Sears at 41% among likely voters ([roanoke.edu](https://www.roanoke.edu/news/rc_poll_politics_october_2025?utm_source=openai)).

Aggregate Polling:

- According to RealClearPolitics and other aggregate tracking sources, as of early November, Spanberger consistently leads by an average of about **52.5% to 42.8%**, a near **+9.7 point margin** ([en.wikipedia.org](https://en.wikipedia.org/wiki/2025_Virginia_gubernatorial_election?utm_source=openai)).

Election Outcome:

- On **November 4, 2025**, Abigail Spanberger won the gubernatorial election decisively with **57.36%** of the vote to Winsome Earle‑Sears’ **42.44%** ([en.wikipedia.org](https://en.wikipedia.org/wiki/2025_Virginia_gubernatorial_election?utm_source=openai)).

Summary:

Across multiple independent polls throughout 2025, **Abigail Spanberger consistently led over Winsome Earle‑Sears**—commonly by **7 to 12 percentage points**. These leads were sustained up to the election, and in the actual vote, Spanberger won by roughly **15 points**, making history as Virginia’s **first female governor**.

Let me know if you’d like a breakdown of polling trends over time, demographic analysis, or how the lieutenant governor’s race compared.

In [ ]:
combined_search_response = client.responses.create(
    model="gpt-4o",  # or another supported model
    input= query,
    temperature = 0,
    instructions="Retrieve the results from the file search first, and use the web search tool to expand the results with news resources",
    tools=[{
        "type": "file_search",
        "vector_store_ids": [vector_store_id],
    },
        {
            "type": "web_search"
        }
    ]
)